# Install kaggle-environments

In [ ]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
# !curl -X PURGE https://pypi.org/simple/kaggle-environments

# Halite environment was defined in v0.2.1
!pip install 'kaggle-environments>=0.2.1'

# Create Halite Environment

In [ ]:
from kaggle_environments import evaluate, make

env = make("halite", debug=True)
env.render()

# Create a Submission (agent)

In [ ]:
%%writefile submission.py
# for Debug previous line (%%writefile submission.py) should be commented out, uncomment to write submission.py

#FUNCTIONS###################################################
def get_map(obs):
    """ get map as two dimensional array of objects and set amounts of halite in each cell """
    game_map = []
    for x in range(conf.size):
        game_map.append([])
        for y in range(conf.size):
            game_map[x].append({
                # value will be ID of owner
                "shipyard": None,
                # value will be ID of owner
                "ship": None,
                # value will be amount of halite
                "ship_cargo": None,
                # amount of halite
                "halite": obs.halite[conf.size * y + x]
            })
    return game_map

def get_vortex_units_coords_and_update_map(v_env):
    """ get lists of coords of Vortex's units and update locations of ships and shipyards on the map """
    # arrays of (x, y) coords
    vortex_shipyards_coords = []
    vortex_ships_coords = []
    # place on the map locations of units of every player
    for player in range(len(v_env["obs"].players)):
        # place on the map locations of every shipyard of the player
        shipyards = list(v_env["obs"].players[player][1].values())
        for shipyard in shipyards:
            x = shipyard % conf.size
            y = shipyard // conf.size
            # place shipyard on the map
            v_env["map"][x][y]["shipyard"] = player
            if player == v_env["obs"].player:
                vortex_shipyards_coords.append((x, y))
        # place on the map locations of every ship of the player
        ships = list(v_env["obs"].players[player][2].values())
        for ship in ships:
            x = ship[0] % conf.size
            y = ship[0] // conf.size
            # place ship on the map
            v_env["map"][x][y]["ship"] = player
            v_env["map"][x][y]["ship_cargo"] = ship[1]
            if player == v_env["obs"].player:
                vortex_ships_coords.append((x, y))
    return vortex_shipyards_coords, vortex_ships_coords

def get_c(c):
    """ get coordinate, considering donut type of the map """
    return c % conf.size

def get_directions(i0, i1, i2, i3):
    """ get list of directions in a certain sequence """
    return [directions_list[i0], directions_list[i1], directions_list[i2], directions_list[i3]]

def clear(x, y, player, game_map, cargo):
    """ check if cell is safe to move in """
    # if there is no ship or hostile ship has less or equal amount of halite
    if (game_map[x][y]["ship"] == None or
            (game_map[x][y]["ship"] != player and game_map[x][y]["ship_cargo"] > cargo)):
        return True
    return False

def define_tiers_corners():
    """ define coordinates of corners for every tier  """
    for tier in tiers:
        tier["corners_coords"]["NE"]["x"] = get_c(vortex_center["x"] + tier["to_center"])
        tier["corners_coords"]["NE"]["y"] = get_c(vortex_center["y"] - tier["to_center"])
        tier["corners_coords"]["SE"]["x"] = get_c(vortex_center["x"] + tier["to_center"])
        tier["corners_coords"]["SE"]["y"] = get_c(vortex_center["y"] + tier["to_center"])
        tier["corners_coords"]["SW"]["x"] = get_c(vortex_center["x"] - tier["to_center"])
        tier["corners_coords"]["SW"]["y"] = get_c(vortex_center["y"] + tier["to_center"])
        tier["corners_coords"]["NW"]["x"] = get_c(vortex_center["x"] - tier["to_center"])
        tier["corners_coords"]["NW"]["y"] = get_c(vortex_center["y"] - tier["to_center"])
        
def at_tier_corner(x, y, tier):
    """ check if ship is at tier's corner """
    for c in tier["corners_coords"]:
        if x == tier["corners_coords"][c]["x"] and y == tier["corners_coords"][c]["y"]:
            return True
    return False

def define_some_globals(configuration):
    """ define some of the global variables """
    global conf
    global low_amount_of_halite
    global min_halite_to_unload
    global globals_not_defined
    conf = configuration
    low_amount_of_halite = conf.maxCellHalite * 0.75
    min_halite_to_unload = conf.spawnCost
    globals_not_defined = False

def adapt_environment(observation, configuration):
    """ adapt environment for the Vortex """
    v_env = {}
    v_env["obs"] = observation
    v_env["ships_keys"] = list(v_env["obs"].players[v_env["obs"].player][2].keys())
    v_env["ships_values"] = list(v_env["obs"].players[v_env["obs"].player][2].values())
    v_env["shipyards_keys"] = list(v_env["obs"].players[v_env["obs"].player][1].keys())
    if globals_not_defined:
        define_some_globals(configuration)
        vortex_center["x"] = v_env["ships_values"][0][0] % conf.size
        vortex_center["y"] = v_env["ships_values"][0][0] // conf.size
        define_tiers_corners()
    v_env["map"] = get_map(v_env["obs"])
    v_env["vortex_halite"] = v_env["obs"].players[v_env["obs"].player][0]
    v_env["vortex_shipyards_coords"], v_env["vortex_ships_coords"] = get_vortex_units_coords_and_update_map(v_env)
    # reset current amounts of ships in tiers
    for tier in tiers:
        tier["ships_amount"] = 0
    # calculate current amounts of ships in tiers
    for i in range(len(v_env["ships_keys"])):
        if v_env["ships_keys"][i] in ships_data:
            tiers[ships_data[v_env["ships_keys"][i]]["tier"]]["ships_amount"] += 1
    return v_env
    
def actions_of_ships(v_env):
    """ actions of every ship in the Vortex """
    actions = {}
    for i in range(len(v_env["ships_keys"])):
        x = v_env["vortex_ships_coords"][i][0]
        y = v_env["vortex_ships_coords"][i][1]
        ship_key = v_env["ships_keys"][i]
        # if this is a new ship
        if ship_key not in ships_data:
            for tier in tiers:
                if tier["ships_amount"] < tier["ships_max_amount"]:
                    ships_data[ship_key] = {
                        # ship is being assigned to this tier
                        "tier": tier["number"],
                        # list of directions to move through
                        "directions": movement_tactics[tier["tactics_index"]]["directions"],
                        "directions_index": 0,
                        # ship is currently moving to its tier
                        "moving_to_tier": True,
                        # ship is currently moving to shipyard
                        "moving_to_shipyard": False,
                        # ship is currently at its tier
                        "at_tier": False
                    }
                    tier["ships_amount"] += 1
                    break
        ship = ships_data[ship_key]
        # if it is last step
        if (v_env["obs"].step == (conf.episodeSteps - 2) and
                v_env["ships_values"][i][1] >= conf.convertCost):
            actions[ship_key] = "CONVERT"
            v_env["map"][x][y]["ship"] = None
            tiers[ship["tier"]]["ships_amount"] -= 1
            continue
        # if ship is at vortex center
        if vortex_center["x"] == x and vortex_center["y"] == y:
            ship["moving_to_shipyard"] = False
            # if there is no shipyard and enough halite to create one
            if (v_env["map"][x][y]["shipyard"] == None and
                    (v_env["vortex_halite"] + v_env["ships_values"][i][1]) >= conf.convertCost):
                actions[ship_key] = "CONVERT"
                v_env["map"][x][y]["ship"] = None
                tiers[ship["tier"]]["ships_amount"] -= 1
                continue
            # else move to ship's tier
            else:
                ship["moving_to_tier"] = True
                ship["directions_index"] = 0
        # if ship is moving to its tier
        if ship["moving_to_tier"]:
            # move north
            x_next = directions_list[0]["x"](x)
            y_next = directions_list[0]["y"](y)
            if clear(x_next, y_next, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                actions[ship_key] = directions_list[0]["direction"]
                v_env["map"][x][y]["ship"] = None
                v_env["map"][x_next][y_next]["ship"] = v_env["obs"].player
                if y_next == tiers[ship["tier"]]["corners_coords"]["NE"]["y"]:
                    ship["moving_to_tier"] = False
                    ship["at_tier"] = True
        # if ship is moving to shipyard
        elif ship["moving_to_shipyard"] != False:
            x_next = ship["moving_to_shipyard"]["x"](x)
            y_next = ship["moving_to_shipyard"]["y"](y)
            if clear(x_next, y_next, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                actions[ship_key] = ship["moving_to_shipyard"]["direction"]
                v_env["map"][x][y]["ship"] = None
                v_env["map"][x_next][y_next]["ship"] = v_env["obs"].player
        # if ship is at its tier and harvesting is not possible
        elif (ship["at_tier"] and
                (not tiers[ship["tier"]]["harvesting"] or v_env["map"][x][y]["halite"] <= low_amount_of_halite)):
            # if ship has enough halite to unload it at shipyard
            if (v_env["ships_values"][i][1] > min_halite_to_unload and (y == vortex_center["y"] or
                    (x == vortex_center["x"] and y == tiers[ship["tier"]]["corners_coords"]["SE"]["y"]))):
                if y == vortex_center["y"]:
                    # if ship is at eastern border of the tier
                    if x == tiers[ship["tier"]]["corners_coords"]["SE"]["x"]:
                        # move west
                        x_next = directions_list[3]["x"](x)
                        if clear(x_next, y, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                            ship["moving_to_shipyard"] = directions_list[3]
                            actions[ship_key] = ship["moving_to_shipyard"]["direction"]
                            v_env["map"][x][y]["ship"] = None
                            v_env["map"][x_next][y]["ship"] = v_env["obs"].player
                            ship["at_tier"] = False
                    # if ship is at western border of the tier
                    elif x == tiers[ship["tier"]]["corners_coords"]["SW"]["x"]:
                        # move east
                        x_next = directions_list[1]["x"](x)
                        if clear(x_next, y, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                            ship["moving_to_shipyard"] = directions_list[1]
                            actions[ship_key] = ship["moving_to_shipyard"]["direction"]
                            v_env["map"][x][y]["ship"] = None
                            v_env["map"][x_next][y]["ship"] = v_env["obs"].player
                            ship["at_tier"] = False
                # if ship is at southern border of the tier
                elif x == vortex_center["x"] and y == tiers[ship["tier"]]["corners_coords"]["SE"]["y"]:
                    # move north
                    y_next = directions_list[0]["y"](y)
                    if clear(x, y_next, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                        ship["moving_to_shipyard"] = directions_list[0]
                        actions[ship_key] = ship["moving_to_shipyard"]["direction"]
                        v_env["map"][x][y]["ship"] = None
                        v_env["map"][x][y_next]["ship"] = v_env["obs"].player
                        ship["at_tier"] = False
            else:
                directions_index = ship["directions_index"]
                if at_tier_corner(x, y, tiers[ship["tier"]]):
                    directions_index += 1
                    if directions_index >= len(ship["directions"]):
                        directions_index = 0
                x_next = ship["directions"][directions_index]["x"](x)
                y_next = ship["directions"][directions_index]["y"](y)
                if clear(x_next, y_next, v_env["obs"].player, v_env["map"], v_env["map"][x][y]["ship_cargo"]):
                    ship["directions_index"] = directions_index
                    actions[ship_key] = ship["directions"][ship["directions_index"]]["direction"]
                    v_env["map"][x][y]["ship"] = None
                    v_env["map"][x_next][y_next]["ship"] = v_env["obs"].player
    return actions
     
def action_of_shipyard(actions, v_env):
    """ action of the Vortex's shipyard """
    # if there is a shipyard
    if v_env["map"][vortex_center["x"]][vortex_center["y"]]["shipyard"] == v_env["obs"].player:
        # if there is enough halite and shipyard is empty
        if (v_env["vortex_halite"] >= conf.spawnCost and
                v_env["map"][vortex_center["x"]][vortex_center["y"]]["ship"] == None):
            # if there is a place for a ship in some tier
            for tier in tiers:
                if tier["ships_amount"] < tier["ships_max_amount"]:
                    tier["ships_amount"] += 1
                    v_env["vortex_halite"] -= conf.spawnCost
                    actions[v_env["shipyards_keys"][0]] = "SPAWN"
                    v_env["map"][vortex_center["x"]][vortex_center["y"]]["ship"] = v_env["obs"].player
    return actions


#GLOBAL_VARIABLES#############################################
conf = None
# coordinates of Vortex's center
vortex_center = {"x": None, "y": None}
# amount of halite that is considered to be too low for harvesting
low_amount_of_halite = None
# minimal amount of halite to unload at shipyard
min_halite_to_unload = None
# object with ship ids and their data
ships_data = {}
# not all global variables are defined
globals_not_defined = True

# list of directions
directions_list = [
    {
        "direction": "NORTH",
        "x": lambda z: z,
        "y": lambda z: get_c(z - 1)
    },
    {
        "direction": "EAST",
        "x": lambda z: get_c(z + 1),
        "y": lambda z: z
    },
    {
        "direction": "SOUTH",
        "x": lambda z: z,
        "y": lambda z: get_c(z + 1)
    },
    {
        "direction": "WEST",
        "x": lambda z: get_c(z - 1),
        "y": lambda z: z
    }
]

# list of movement tactics
movement_tactics = [
    # W -> S -> E -> N
    {"directions": get_directions(3, 2, 1, 0)},
    # E -> S -> W -> N
    {"directions": get_directions(1, 2, 3, 0)}
]
movement_tactics_amount = len(movement_tactics)

# vortex tiers and their properties
tiers = [
    {
        # number of the tier
        "number": 0,
        # distance to vortex center
        "to_center": 1,
        # maximum amount of ships on this tier
        "ships_max_amount": 1,
        # current amount of ships on this tier
        "ships_amount": 0,
        # index of movement tactics
        "tactics_index": 0,
        # coordinates of tier's borders
        "corners_coords": {"NE": {}, "SE": {}, "SW": {}, "NW": {}},
        # is halite harvesting allowed
        "harvesting": True
    },
    {
        # number of the tier
        "number": 1,
        # distance to vortex center
        "to_center": 2,
        # maximum amount of ships on this tier
        "ships_max_amount": 3,
        # current amount of ships on this tier
        "ships_amount": 0,
        # index of movement tactics
        "tactics_index": 1,
        # coordinates of tier's borders
        "corners_coords": {"NE": {}, "SE": {}, "SW": {}, "NW": {}},
        # is halite harvesting allowed
        "harvesting": True
    },
    {
        # number of the tier
        "number": 2,
        # distance to vortex center
        "to_center": 3,
        # maximum amount of ships on this tier
        "ships_max_amount": 5,
        # current amount of ships on this tier
        "ships_amount": 0,
        # index of movement tactics
        "tactics_index": 0,
        # coordinates of tier's borders
        "corners_coords": {"NE": {}, "SE": {}, "SW": {}, "NW": {}},
        # is halite harvesting allowed
        "harvesting": True
    },
    {
        # number of the tier
        "number": 3,
        # distance to vortex center
        "to_center": 4,
        # maximum amount of ships on this tier
        "ships_max_amount": 9,
        # current amount of ships on this tier
        "ships_amount": 0,
        # index of movement tactics
        "tactics_index": 1,
        # coordinates of tier's borders
        "corners_coords": {"NE": {}, "SE": {}, "SW": {}, "NW": {}},
        # is halite harvesting allowed
        "harvesting": True
    }
]


#THE_VORTEX####################################################
def vortex_agent(observation, configuration):
    """ INTO THE VORTEX!!! """
    v_env = adapt_environment(observation, configuration)
    actions = actions_of_ships(v_env)
    actions = action_of_shipyard(actions, v_env)
    return actions

# Debug your Agent

In [ ]:
if "vortex_agent" in globals():
    # reset variables
    ships_data = {}
    globals_not_defined = True

    # Play as first position against random agent.
    trainer = env.train([None, "random"])

    observation = trainer.reset()

    while not env.done:
        my_action = vortex_agent(observation, env.configuration)
        print("Step: {0}, My Action: {1}".format(observation.step, my_action))
        observation, reward, done, info = trainer.step(my_action)
        # env.render(mode="ipython", width=100, height=90, header=False, controls=False)
    env.render()

# Evaluate your Agent

In [ ]:
def mean_reward(rewards):
    wins = 0
    ties = 0
    loses = 0
    for r in rewards:
        r0 = 0 if r[0] is None else r[0]
        r1 = 0 if r[1] is None else r[1]
        if r0 > r1:
            wins += 1
        elif r1 > r0:
            loses += 1
        else:
            ties += 1
    return f'wins={wins/len(rewards)}, ties={ties/len(rewards)}, loses={loses/len(rewards)}'

# Run multiple episodes to estimate its performance.
# Setup agentExec as LOCAL to run in memory (runs faster) without process isolation.
print("Vortex Agent vs Random Agent:", mean_reward(evaluate(
    "halite",
    ["submission.py", "random", "random", "random"],
    num_episodes=10, configuration={"agentExec": "LOCAL"}
)))

# Test your Agent

In [ ]:
#env.run(["submission.py", "submission.py", "submission.py", "submission.py"])
env.run(["submission.py", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)

# Submit to Competition

1. Commit this kernel.
2. View the commited version.
3. Go to "Data" section and find submission.py file.
4. Click "Submit to Competition"
5. Go to [My Submissions](https://kaggle.com/c/halite/submissions) to view your score and episodes being played.